# MinerU 2.5 with FiftyOne - Document Parsing Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/mineru_2_5/blob/main/mineru_fiftyone_example.ipynb)

This notebook demonstrates how to use the MinerU 2.5 model with FiftyOne for document parsing tasks.


## Installation

Install required packages:


In [ ]:
%pip install fiftyone
%pip install "mineru-vl-utils[transformers]"
%pip install transformers


## Load Dataset

We'll use a sample dataset from Hugging Face Hub:


In [ ]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

# Load the dataset
dataset = load_from_hub(
    "harpreetsahota/NutriGreen",
    overwrite=True,
    persistent=True,
    name="NutriGreen_MinerU25_Outputs",
    max_samples=5
)


## Register and Load MinerU 2.5 Model

Register the model source and load the model:


In [ ]:
import fiftyone.zoo as foz

# Register the model source
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/mineru_2_5",
    overwrite=True
)

# Load the model
model = foz.load_zoo_model("opendatalab/MinerU2.5-2509-1.2B")


## Apply OCR Detection (Structured Extraction)

Extract document structure with bounding boxes:


In [ ]:
# Set operation to OCR detection
model.operation = "ocr_detection"

# Apply model to dataset
dataset.apply_model(model, label_field="text_detections")

print(f"Applied OCR detection to {len(dataset)} samples")


## Apply OCR (Plain Text Extraction)

Extract plain text content:


In [ ]:
# Set operation to OCR
model.operation = "ocr"

# Apply model to dataset
dataset.apply_model(model, label_field="text_extraction")

print(f"Applied OCR extraction to {len(dataset)} samples")


## Visualize Results

Launch the FiftyOne App to visualize the results:


In [ ]:
# Launch the FiftyOne App
session = fo.launch_app(dataset)


## Inspect Sample Results

Let's look at the results from one sample:


In [ ]:
# Get the first sample
sample = dataset.first()

# Print detection results
print("=== Detections ===")
if sample.text_detections:
    for detection in sample.text_detections.detections[:3]:  # Show first 3
        print(f"Type: {detection.label}")
        print(f"BBox: {detection.bounding_box}")
        print(f"Text: {detection.text[:100]}...")  # First 100 chars
        print("---")

# Print OCR text
print("\n=== OCR Text (first 500 chars) ===")
if sample.text_extraction:
    print(sample.text_extraction[:500])
